In [2]:
!pip install psycopg2-binary



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 11.9 MB/s eta 0:00:0031m12.6 MB/s eta 0:00:01


In [3]:
import pandas as pd
import psycopg2


In [71]:
# Step 2: Connect to the Database
import psycopg2
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="a9Am7Yy5r9_T7h4OF2GN",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor() 
print("✅ Connection successful!")


✅ Connection successful!


In [51]:
# Step 3: Run a Test Query
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head() 


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/890545935.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [52]:
# Step 4: Count schools by borough
query = """
SELECT borough, COUNT(*) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY school_count DESC;
"""
df_distribution = pd.read_sql(query, conn)
df_distribution


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/3216484643.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_distribution = pd.read_sql(query, conn)


,borough,school_count
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


In [53]:
# Step 5 : Average ELL% per borough
query = """
SELECT d.borough, ROUND(AVG(s.ell_percent)) AS avg_ell_percent
FROM nyc_schools.school_demographics AS s
JOIN nyc_schools.high_school_directory AS d
ON s.dbn = d.dbn
GROUP BY d.borough
ORDER BY avg_ell_percent DESC;
"""
df_ell = pd.read_sql(query, conn)
df_ell





/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/1612687833.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Manhattan,8.0


In [55]:
query = "SELECT DISTINCT borough FROM nyc_schools.high_school_directory;"
df_boroughs = pd.read_sql(query, conn)
df_boroughs


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/2892404296.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_boroughs = pd.read_sql(query, conn)


,borough
0,Brooklyn
1,Queens
2,Staten Island
3,Manhattan
4,Bronx


In [56]:
query = "SELECT DISTINCT dbn FROM nyc_schools.school_demographics;"
df_dbns = pd.read_sql(query, conn)
len(df_dbns)


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/538210552.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dbns = pd.read_sql(query, conn)


32

In [57]:
query = """
SELECT d.borough, ROUND(AVG(s.ell_percent)) AS avg_ell_percent
FROM nyc_schools.high_school_directory AS d
LEFT JOIN nyc_schools.school_demographics AS s
ON s.dbn = d.dbn
GROUP BY d.borough
ORDER BY avg_ell_percent DESC;
"""
df_ell = pd.read_sql(query, conn)
df_ell


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/2381284048.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Brooklyn,NaN
1,Queens,NaN
2,Staten Island,NaN
3,Bronx,NaN
4,Manhattan,8.0


In [19]:
query = "SELECT * FROM nyc_schools.school_safety_report LIMIT 5;"
df_check = pd.read_sql(query, conn)
df_check.columns


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/614300381.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_check = pd.read_sql(query, conn)


Index(['school_year', 'building_code', 'dbn', 'location_name', 'location_code',
       'address', 'borough', 'geographical_district_code', 'register',
       'building_name', 'num_schools', 'schools_in_building', 'major_n',
       'oth_n', 'nocrim_n', 'prop_n', 'vio_n', 'engroupa', 'rangea',
       'avgofmajor_n', 'avgofoth_n', 'avgofnocrim_n', 'avgofprop_n',
       'avgofvio_n', 'borough_name', 'postcode', 'latitude', 'longitude',
       'community_board', 'council_district', 'census_tract', 'bin', 'bbl',
       'nta', '_schools'],
      dtype='object')

In [21]:
list(df_check.columns)


['school_year',
 'building_code',
 'dbn',
 'location_name',
 'location_code',
 'address',
 'borough',
 'geographical_district_code',
 'register',
 'building_name',
 'num_schools',
 'schools_in_building',
 'major_n',
 'oth_n',
 'nocrim_n',
 'prop_n',
 'vio_n',
 'engroupa',
 'rangea',
 'avgofmajor_n',
 'avgofoth_n',
 'avgofnocrim_n',
 'avgofprop_n',
 'avgofvio_n',
 'borough_name',
 'postcode',
 'latitude',
 'longitude',
 'community_board',
 'council_district',
 'census_tract',
 'bin',
 'bbl',
 'nta',
 '_schools']

In [58]:
df_check.head(3)


,school_year,building_code,dbn,location_name,location_code,address,borough,geographical_district_code,register,building_name,...,postcode,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,_schools
0,2015-16,K006,17K006,P.S. 006,K006,43 SNYDER AVENUE,K,17.0,712,,...,11226.0,40.65,-73.96,14.0,40.0,794.0,3.12e+06,3.05e+09,Erasmus ...,1
1,2015-16,K005,16K005,P.S. 005 Dr. Ronald Mcnair,K005,820 HANCOCK STREET,K,16.0,231,,...,11233.0,40.69,-73.92,3.0,41.0,377.0,3.04e+06,3.01e+09,Stuyvesant Heights ...,1
2,2015-16,K003,13K003,P.S. 003 The Bedford Village,K003,50 JEFFERSON AVENUE,K,13.0,450,,...,11216.0,40.68,-73.96,3.0,36.0,227.0,3.06e+06,3.02e+09,Clinton Hill ...,1


In [59]:
[col for col in df_check.columns if "major" in col.lower()]


['major_n', 'avgofmajor_n']

In [60]:
# Step 6 (Final) :
query = """
SELECT d.borough,
       AVG(s.avgofmajor_n) AS avg_major_incidents
FROM nyc_schools.school_safety_report AS s
JOIN nyc_schools.high_school_directory AS d
ON s.dbn = d.dbn
GROUP BY d.borough
ORDER BY avg_major_incidents DESC;
"""

df_safety = pd.read_sql(query, conn)
df_safety


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/44751524.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_safety = pd.read_sql(query, conn)


,borough,avg_major_incidents
0,Staten Island,1.01
1,Queens,0.89
2,Brooklyn,0.69
3,Manhattan,0.54
4,Bronx,0.48


In [61]:
query = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
pd.read_sql(query, conn)


/var/folders/mn/w43_rvg92n9dv58q76z80hgc0000gn/T/ipykernel_16947/3396966052.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(query, conn)


,table_name
0,mytable_martin
1,users
2,sat_results
3,sat_scores
4,departments
5,hospitals_refactored_test
6,mytable
7,hourly_metrics
8,geography_columns
9,geometry_columns


In [75]:
# ==== FINAL RESULTS SUMMARY ====
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2)

print("✅ Final Results Summary")

# Schools per Borough
print("\n— Schools per Borough —")
try:
    display(df_schools)
except NameError:
    print("df_schools not found. (Run the schools-per-borough cell.)")

# Average ELL% per Borough
print("\n— Average ELL% per Borough —")
try:
    display(df_ell_nulls)
except NameError:
    print("df_ell_nulls not found. (Run the ELL cell with LEFT JOIN.)")

# Average Major Incidents per Borough
print("\n— Average Major Incidents per Borough —")
try:
    display(df_safety)
except NameError:
    print("df_safety not found. (Run the safety/major incidents cell.)") 


✅ Final Results Summary

— Schools per Borough —


,borough,total_schools
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10



— Average ELL% per Borough —


,borough,avg_ell_percent
0,None,12.71
1,Manhattan,7.57



— Average Major Incidents per Borough —


,borough,avg_major_incidents
0,Staten Island,1.01
1,Queens,0.89
2,Brooklyn,0.69
3,Manhattan,0.54
4,Bronx,0.48


 📘 Notes on Data Availability
1- The ELL (English Language Learner) data is only available for Manhattan in the current dataset.
2- Other boroughs show NULL values because their demographic records are missing.
3- This indicates a data gap rather than an error in the queries or the notebook.
